In [6]:
import os
import sys

from libs.community.langchain_community.tools.azure_ai_services.translate_tool import AzureTranslateTool
import langchain_core.agents as agents



TypeError: cannot pickle 'classmethod' object

In [7]:
# Set up environment variables
translate_key = os.getenv("AZURE_TRANSLATE_API_KEY")
translate_endpoint = os.getenv("AZURE_TRANSLATE_ENDPOINT")
region = "westus2"  # Specify your Azure region

# Instantiate the AzureTranslateTool
translate_tool = AzureTranslateTool(
    text_translation_key=translate_key,
    text_translation_endpoint=translate_endpoint,
    region=region,
)


NameError: name 'AzureTranslateTool' is not defined

In [4]:
def translate_text(input_text):
    # Create a HumanMessage for the input
    human_message = agents.HumanMessage(content=input_text)
    
    # Translate the input text using the AzureTranslateTool
    translated_text = translate_tool._run(human_message.content, to_language='pt')  
    
    # Create an AIMessage for the output
    ai_message = agents.AIMessage(content=f"Translated text: {translated_text}")
    
    return ai_message



In [5]:
# Simulate user input
user_input = "hola mundo"  # 

# Call the translation function
response = translate_text(user_input)

# Print the AI's response
print(response.content) 



NameError: name 'agents' is not defined